In [13]:
''' Importing all the required modules '''
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import pickle
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [14]:
def Calibration_function():
    '''Detects distortion cofficients by analysing few test images from the source camera. These Distortion coefficients 
    are used to undistort new input images to the software pipeline. The Camera Calibration should be performed
    only once before start of final execution.'''
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('Calibration_Data/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    ############ Using the Calibration Parameters for Distortion Correction
    img      = cv2.imread('Calibration_Data/test1.jpg', cv2.IMREAD_COLOR)
    img_size = (img.shape[1], img.shape[0])
    
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
        
    return mtx, dist

In [15]:
def ImageTransformation_function(main_image):
    '''Processing the input image to detect the main object of interest and highlights the detected objects in the final output
    A color space based transformation function is used over Canny or Sobel due to the high robustness in detection accuracy
    under stress conditions like low lighting, noise ratio, etc.
    '''
    ############ LAB Image Processing -- Yellow Lanes
    lab_image    = cv2.cvtColor(main_image , cv2.COLOR_RGB2LAB)
    L,A,B=cv2.split(lab_image)
    B_image = B
    ret,whitelane_image = cv2.threshold(B_image,160,200,cv2.THRESH_BINARY)
   
    ############# RGB Image Processing -- White Lanes   
    lower = np.array([225,180,200],dtype = "uint8")
    upper = np.array([255, 255, 255],dtype = "uint8")
    mask = cv2.inRange(main_image , lower, upper)
    S = cv2.bitwise_and(main_image, main_image, mask = mask)
    S = cv2.cvtColor(S, cv2.COLOR_RGB2GRAY)
    yellowlane_image = S
    ret,S_binary = cv2.threshold(yellowlane_image,160,200,cv2.THRESH_BINARY)
    
    ############ Combining HLS and LAB outputs
    combined_image =  cv2.bitwise_or(yellowlane_image,whitelane_image)
    
    return combined_image

In [16]:
def PerspectiveTransform_function(img):
    '''Takes the input image and creates a bird's eye view of the image with only the main object of interest in the scope. ''' 
    offset = 100 # offset for dst points
    
    # Grab the image shape
    img_size = (img.shape[1], img.shape[0])
    
    leftupperpoint  = [568,470]
    rightupperpoint = [717,470]
    leftlowerpoint  = [260,680]
    rightlowerpoint = [1043,680]

    src = np.float32([leftupperpoint, leftlowerpoint, rightupperpoint, rightlowerpoint])
    dst = np.float32([[200,0], [200,680], [1000,0], [1000,680]])

    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    M_inv = cv2.getPerspectiveTransform(dst, src)

    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_NEAREST)
   
    return warped, M, M_inv

In [17]:
def Directsearch_function(binary_warped):
    '''Directly searches for the Lane markings without considering the previous observations, and returns the location of the
    lane markings on the current image. This is a more calculation intensive function.'''
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 10
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 80 

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low       = binary_warped.shape[0] - (window+1)*window_height
        win_y_high      = binary_warped.shape[0] - window*window_height

        win_xleft_low   = leftx_current  - margin
        win_xleft_high  = leftx_current  + margin
        win_xright_low  = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]

        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))

        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        
    except ValueError:
        # Avoids an error if the above is not implemented fully
        print("Value Error Received")
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty


In [18]:
def Indirectsearch_function(binary_warped,left_fit,right_fit):
    '''Based on the previous observations, searches for the lane markings near the previous lane points. This function is less
    calculation intensive. '''
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    margin = 100
    
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]    
    
    return leftx, lefty, rightx, righty

In [19]:
def SanityCheck_function(left_fit,right_fit):
    '''Checks whether tha lane detection by Indirect/Direct conditions were successfull.'''
    #Status = True  --> Valid Line
    #Status = False --> Invalid Line
    status = True
    try:
        if len(left_fit) ==0 or len(right_fit) == 0:
            status = False
    except TypeError:
        status = False
        
    return status


In [20]:
def LaneAproximation_function(binary_warped, LaneEquations, Sanity_status):
    '''Calls Direct/Indirect Search functions to find lanes in the input image. The selection of the Direct/Indirect function
    depends on SanityCheck status from the previous detection results'''
    if(not Sanity_status):
        leftx, lefty, rightx, righty = Directsearch_function(binary_warped)
    else:
        leftx, lefty, rightx, righty = Indirectsearch_function(binary_warped, LaneEquations[2], LaneEquations[3])
        
        try:
            left_fit  = np.polyfit(lefty, leftx, 2)
            right_fit = np.polyfit(righty, rightx, 2)
            Sanity_status =  SanityCheck_function(left_fit,right_fit) 
            
            if(not Sanity_status):
                leftx, lefty, rightx, righty = Directsearch_function(binary_warped)
                
        except TypeError:
            
            #Using Previous Values
            left_fit  = LaneEquations[2] 
            right_fit = LaneEquations[3]    
    
    out_img = np.dstack((binary_warped,binary_warped,binary_warped))
    
    # Fit a second order polynomial to each using `np.polyfit`
    try:
        left_fit  = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    except TypeError:
        #Using Previous Values
        left_fit  = LaneEquations[2] 
        right_fit = LaneEquations[3]

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx  = left_fit[0] *ploty**2 + left_fit[1] *ploty +  left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx]   = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]    

    Sanity_status =  SanityCheck_function(left_fit,right_fit) 

    return out_img, [left_fitx, right_fitx, left_fit, right_fit], Sanity_status

In [21]:
def radius_curvature(binary_warped, left_fit, right_fit):
    '''Calculates the Right/Left Curvature and Vehicle offset based on the lane detection results.'''
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    
    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    
    # Calculate the new radii of curvature
    Left_CurvatureRadius  =  ((1 + (2*left_fit_cr[0] *y_eval*ym_per_pix + left_fit_cr[1])**2) **1.5) / np.absolute(2*left_fit_cr[0])
    Right_CurvatureRadius = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    # Calculate vehicle center
    #left_lane and right lane bottom in pixels
    Left_LaneBottom = (left_fit[0]*y_eval)**2 + left_fit[0]*y_eval + left_fit[2]
    Right_LaneBottom = (right_fit[0]*y_eval)**2 + right_fit[0]*y_eval + right_fit[2]
    
    # Lane center as mid of left and right lane bottom                        
    lane_center = (Left_LaneBottom + Right_LaneBottom)/2.
    center_image = 640
    center = (lane_center - center_image)*xm_per_pix #Convert to meters
    position = "left" if center < 0 else "right"
    center = "Vehicle is {:.2f}m {}".format(center, position)
    
    # Now our radius of curvature is in meters
    return Left_CurvatureRadius, Right_CurvatureRadius, center

In [22]:
def draw_on_image(undist, warped_img, left_fit, right_fit, Minv, left_curvature, right_curvature, center):
    '''Takes all the detection parameters like Lane markings and curvation/offset details calculated from the helper functions
    and overlays it over each input frame'''
    ploty = np.linspace(0, warped_img.shape[0]-1, warped_img.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped_img).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
    cv2.putText(result, 'Left curvature: {:.0f} m'.format(left_curvature), (50, 50), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, 'Right curvature: {:.0f} m'.format(right_curvature), (50, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
    cv2.putText(result, '{}'.format(center), (50, 150), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 255, 255), 2)
        
    return result

In [23]:
'''All the Helper functions are invoked using a callable class which initializes once to calculate the calibration parameters,
then subsequently runs the call section which runs on each input frame.'''
class DetectionPipeline:
    def __init__(self):
        self.mtx , self.dist = Calibration_function()
        #LaneSearch_flag = True  --> Direct Lane Search
        #LaneSearch_flag = False --> InDirect Lane Search
        self.Sanity_status = False
        self.LaneEquations = None
               
    def __call__(self, main_image):
        UndistortedImage   = cv2.undistort(main_image, self.mtx, self.dist, None, self.mtx)   
        Transformed_Image  = ImageTransformation_function(UndistortedImage)
        Birdseye_Image, M, M_inv   = PerspectiveTransform_function(Transformed_Image)
        Lane_Image, self.LaneEquations, self.Sanity_status  = LaneAproximation_function(Birdseye_Image, self.LaneEquations, self.Sanity_status)
        Left_curve, Right_curve, Lane_Center = radius_curvature(Lane_Image, self.LaneEquations[2], self.LaneEquations[3])
        Final_Ouput = draw_on_image(UndistortedImage, Birdseye_Image, self.LaneEquations[2], self.LaneEquations[3], M_inv, Left_curve, Right_curve, Lane_Center)
        
        return Final_Ouput

'''Module Acceptance Test Case'''
#image = cv2.imread('Test_Images/test1.jpg')
#class_image    = cv2.cvtColor(image , cv2.COLOR_BGR2RGB)
DetectionPipeline_Instance = DetectionPipeline()
#plt.imshow(DetectionPipeline_Instance(class_image))

#Running Software Pipeline on a Video Stream
white_output = 'project_video_output.mp4'
clip1 = VideoFileClip("Test_Videos/project_video.mp4").subclip(1,10)
white_clip = clip1.fl_image(DetectionPipeline_Instance)
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 225/226 [00:35<00:00,  5.82it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 16.5 s, sys: 369 ms, total: 16.9 s
Wall time: 38.7 s
